In [ ]:
# === JupyterLab 内部调试版本（线程版本）===
# 启动 server 于后台线程，并支持调试变量与控制

from http_server1 import run_server_debug  # 确保这个函数已定义并可调试
import socket
import os
import threading
import time
from multiprocessing import Process

# === 配置参数 ===
PORT = 8011
HOST = "192.168.50.253"
TEST_FILE = "rfc2616.html"
OUTPUT_FILE = "copy.html"

# === 以线程启动 server，不阻塞主线程 ===
# print(f"[i] Launching run_server_debug({PORT}) in background thread...")
# threading.Thread(target=run_server_debug, args=(PORT,), daemon=True).start()
# time.sleep(2)  # 等待 server 完全启动

# === 以进程启动 server，与Jupyter 调试器进程独立 ===
# server_process = Process(target=run_server_debug, args=(PORT,))
# server_process.start()
# time.sleep(1)  # 等待 server 完全启动

# === 调用 server 函数（可调试） ===
print(f"[i] Calling run_server_debug({PORT}) from Jupyter...")
run_server_debug(PORT)

# === 启动测试 ===
NUM_CLIENTS = 10
OUTPUT_TEMPLATE = "copy_client_{}.html"

def client_task(i):
        try:
            request_line = f"GET /{TEST_FILE} HTTP/1.0\r\nHost: {HOST}\r\n\r\n"
            with socket.create_connection((HOST, PORT), timeout=5) as s:
                s.sendall(request_line.encode())
                response = b""
                while True:
                    chunk = s.recv(4096)
                    if not chunk:
                        break
                    response += chunk

            header_data, body = response.split(b"\r\n\r\n", 1) if b"\r\n\r\n" in response else (b"", response)
            output_file = OUTPUT_TEMPLATE.format(i)
            with open(output_file, "wb") as f:
                f.write(body)
            print(f"[✓] Client {i+1}: saved {output_file}")
        except Exception as e:
            print(f"[!] Client {i+1} failed: {e}")

threads = [threading.Thread(target=client_task, args=(i,), daemon=True) for i in range(NUM_CLIENTS)]
for t in threads: t.start()
for t in threads: t.join()

print("\nAll client requests completed.")